In [2]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import sys
import os
import math
sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
SP_VOCAB_SIZE = 5000
TRAIN_SIZE = 5000

/Users/vik/.virtualenvs/nnets/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from text_data import Opus100DatasetWrapper

class Wrapper(Opus100DatasetWrapper):
    split_lengths = [TRAIN_SIZE, math.floor(TRAIN_SIZE * .1), 100]
    x_length = 40
    target_length = 40

wrapper = Wrapper(SP_VOCAB_SIZE)
datasets = wrapper.generate_datasets(BATCH_SIZE)
train = datasets["train"]
valid = datasets["validation"]

Found cached dataset opus100 (/Users/vik/.cache/huggingface/datasets/opus100/en-es/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704)
100%|██████████| 3/3 [00:00<00:00, 223.38it/s]
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tokens.txt --model_prefix=opus100 --vocab_size=5000 --model_type=unigram
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokens.txt
  input_format: 
  model_prefix: opus100
  model_type: UNIGRAM
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extr

In [4]:
def generate(sequence, pred, target, wrapper):
    prompts = wrapper.decode_batch(sequence.cpu())
    texts = wrapper.decode_batch(torch.argmax(pred, dim=2).cpu())
    correct_texts = wrapper.decode_batch(target.cpu())

    displays = []
    for p, t, ct in zip(prompts, texts, correct_texts):
        displays.append(f"{p} | {ct} | {t}")
    return displays

In [18]:
class TransformerNetwork(nn.Module):
    def __init__(self, input_units, hidden_units, attention_heads, blocks=1):
        super(TransformerNetwork, self).__init__()
        self.transformer = nn.Transformer(hidden_units, attention_heads, num_encoder_layers=blocks, num_decoder_layers=blocks, batch_first=True)
        self.embedding = nn.Embedding(input_units, hidden_units)
        k = math.sqrt(1/hidden_units)
        self.output_embedding = nn.Parameter(torch.rand(hidden_units, input_units) * 2 * k - k)

    def forward(self, x, y):
        embed_x = self.embedding(x)
        embed_y = self.embedding(y)
        output = self.transformer(embed_x, embed_y)
        token_vectors = torch.einsum("bse, eo->bso", output, self.output_embedding)
        return token_vectors

In [27]:
from tqdm.auto import tqdm
import wandb

wandb.init(project="transformer", notes="Pytorch 2 blocks", name="pt-2-blocks")

# TODO: Profile and improve perf - https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html
model = TransformerNetwork(wrapper.vocab_size, 512, 8, blocks=2).to(DEVICE)
loss_fn = nn.CrossEntropyLoss(ignore_index=wrapper.pad_token)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
wandb.watch(model, log_freq=100)

loss,█▆▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,0.01172


[]

In [28]:
EPOCHS = 25
DISPLAY_BATCHES = 2
OUT_SEQUENCE_LEN = wrapper.y_length
PRINT_VALID = True

for epoch in range(EPOCHS):
    # Run over the training examples
    train_loss = 0
    match_pct = 0
    for batch, (sequence, target, prev_target) in tqdm(enumerate(train)):
        optimizer.zero_grad(set_to_none=True)
        pred = model(sequence.to(DEVICE), prev_target.to(DEVICE))

        # If you use a batch, need to reshape pred to be batch * sequence, embedding_len to be compatible
        # Similar reshape with target to be batch * sequence vector of class indices
        loss = loss_fn(pred.reshape(-1, pred.shape[-1]), target.view(-1).to(DEVICE))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    with torch.no_grad():
        mean_loss = train_loss / len(train) / BATCH_SIZE
        wandb.log({"loss": mean_loss})
        print(f"Epoch {epoch} train loss: {mean_loss}")
        sents = generate(sequence, pred, target, wrapper)
        for sent in sents[:DISPLAY_BATCHES]:
            print(sent)

127it [01:09,  1.82it/s]


Epoch 0 train loss: 0.16928739047895267
Nunca he estado mejor. | Never better. | No...
Te vez tan tonta con tus pantaloncitos. | You are so annoying with your little shorts. | Youss.s.s.sss.s


127it [00:56,  2.27it/s]


Epoch 1 train loss: 0.14337334652819972
Posee un módulo de comunicación por radio y la cámara integrada de 1Mpx le permite reconocer visualmente a los usuarios. | It is equipped with a radio communication module as well as an integrated 1Mpx camera, allowing it to visually recognize users. | It  a   a  , , , a  it a  and,       to. be ⁇   to ,,
Ya veo. | Oh, I see. | Oh, I don,


127it [00:52,  2.40it/s]


...

Epoch 7 train loss: 0.07594454939675144
Desde cuando? | Since when? | Since when?
no le pedimos nada a la empresa, porque nosotros nos ocuparemos de Joe. | We ain't asking the company for nothing, cos we're gonna take care of Joe. | We' bu,t theing for ap, theing asing,ts,, a, the,


127it [00:56,  2.24it/s]


Epoch 8 train loss: 0.06672253491958295
¿La palabra clave? | The codeword? | The cowwrawra
Sólo para ponerme a prueba. | - Only to test myself. | - Only....


118it [00:54,  2.15it/s]


KeyboardInterrupt: 